# Part 2: Implementing a CV Research Paper
***
# Table of Contents
## Global
1.   [Setup](#Setup)

## Task A
1.   [Scene Loading](#Task-A-Scene-Loading)
2.   [Impainting](#Task-A-Impainting)
3.   [Result Comparison](#Task-A-Result-Comparison)

## Task B
1.   [Scene Loading](#Task-B-Scene-Loading)
2.   [Mask Handling](#Task-B-Mask-Handling)
3.   [Impainting](#Task-B-Impainting)
4.   [Result Comparison](#Task-B-Result-Comparison)
5.   [Background Subtraction](#Background-Subtraction)

# Setup

3 libraries are used for this part of the assignment:
* [opencv](https://opencv.org/)
* [numpy](https://numpy.org/)
* [tqdm](https://tqdm.github.io/)
* [os](https://docs.python.org/3/library/os.html)

In [1]:
import cv2
print('cv2 version:', cv2.__version__)
import numpy as np
print('numpy version:', np.__version__)
import tqdm
print('tqdm version:', tqdm.__version__)
from tqdm.notebook import tqdm
import os

cv2 version: 4.0.1
numpy version: 1.19.2
tqdm version: 4.59.0


# Task A Scene Loading

In this section the S2, S1 and Mask of 6 object types are loaded in:

* S2 : input image with object/s
* S1 : real image with an object from S2 removed
* Mask : Mask of object to be removed from S2


In [2]:
# Define labels for objects

id_to_Scene = ["Statues",
               "Shooter Glasses",
               "Academic Books",
               "Footwear",
               "Mugs",
               "Tech"]

In [3]:
# Load the S2 scenes

root = 'Images/Task A/S2/'
if not os.access(root, os.R_OK):
    print("Check dataroot!!")

scenes = os.listdir(root)

S2 = [cv2.imread(os.path.join(root, s)) for s in scenes]

In [4]:
# Load the S1 scenes

root = 'Images/Task A/S1/'
if not os.access(root, os.R_OK):
    print("Check dataroot!!")

scenes = os.listdir(root)

S1 = [cv2.imread(os.path.join(root, s)) for s in scenes]


In [5]:
# Load the Masks

root = 'Images/Task A/Masks/'
if not os.access(root, os.R_OK):
    print("Check dataroot!!")

masks = os.listdir(root)

Masks = [cv2.imread(os.path.join(root, m), 0) for m in masks]

# Task A Impainting

Image impainting is the process of adding lost information to an image. It can also be used to remove objects in an image
as shown below.

## TELEA and NS

The 2 methods that are used for impainting as provided by opencv.

In [6]:
impainted = []
for i, img in tqdm(enumerate(S2)):
    impainted.append(cv2.inpaint(img.copy(), Masks[i].copy(), 3, cv2.INPAINT_TELEA))
    cv2.imwrite("Output/Task A TELEA "+ id_to_Scene[i] +".png", impainted[-1], [cv2.IMWRITE_PNG_COMPRESSION, 0])

    impainted.append(cv2.inpaint(img.copy(), Masks[i].copy(), 3, cv2.INPAINT_NS))
    cv2.imwrite("Output/Task A NS "+ id_to_Scene[i] +".png", impainted[-1], [cv2.IMWRITE_PNG_COMPRESSION, 0])

0it [00:00, ?it/s]

# Task A Result Comparison

The SSD and MSE functions from Part 1 are used to compare the S2 impainted result to S1.

In [7]:
def SSD(image1, image2):
    return np.sum((image1 - image2) ** 2)

def MSE(image1, image2):
    return np.mean((image1-image2)**2)

In [8]:
Results = {"SSD": [], "MSE": []}

Results["SSD"] = [SSD(impaint.copy(), S1[i//2].copy()) for i, impaint in enumerate(impainted)]
Results["MSE"] = [MSE(impaint.copy(), S1[i//2].copy()) for i, impaint in enumerate(impainted)]

In [9]:
print(Results)

{'SSD': [116699128, 116998164, 36426554, 36471086, 63658112, 62852151, 48794438, 48595223, 41426087, 41920221, 40899749, 41501292], 'MSE': [42.20888599537037, 42.317044270833335, 13.175113570601852, 13.191220341435185, 23.02449074074074, 22.732982855902776, 17.648451244212964, 17.57639720775463, 14.983393735532408, 15.16211697048611, 14.793022641782407, 15.010594618055556]}


# Task B

# Task B Scene Loading

In [10]:
id_to_Scene = ["Books A NW",
               "Books A W",
               "Bottles A NW",
               "Bottles A W",
               "Electronics A NW",
               "Electronics A W"]

In [11]:
root = 'Images/Task B/S2/'
if not os.access(root, os.R_OK):
    print("Check dataroot!!")

scenes = os.listdir(root)

S2 = [cv2.imread(os.path.join(root, s)) for s in scenes]

In [12]:
root = 'Images/Task B/S1/'
if not os.access(root, os.R_OK):
    print("Check dataroot!!")

scenes = os.listdir(root)

S1 = [cv2.imread(os.path.join(root, s)) for s in scenes]

# Task B Masks

In [13]:
root = 'Images/Task B/Masks/'
if not os.access(root, os.R_OK):
    print("Check dataroot!!")

masks = os.listdir(root)

_Masks = [cv2.imread(os.path.join(root, m), 0) for m in masks]

By checking the matrix we could get specific gray value for each object mask. A new mask is created on this grey value.

In [14]:
greys = [[14], [75, 38], [75], [75], [38], [38]]

Masks = []

for m_count, mask in tqdm(enumerate(_Masks)):
    Masks.append(mask.copy())
    for i, y in enumerate(Masks[-1]):
        for j, x in enumerate(y):
            if len(greys[m_count]) == 1:
                if Masks[-1][i][j] != greys[m_count][0]:
                    Masks[-1][i][j] = 0
                else:
                    Masks[-1][i][j] = 255
            else:
                if Masks[-1][i][j] != greys[m_count][0] and Masks[-1][i][j] != greys[m_count][1]:
                    Masks[-1][i][j] = 0
                else:
                    Masks[-1][i][j] = 255

    cv2.imwrite("Output/Task B Mask "+ str(m_count) +".png", Masks[-1], [cv2.IMWRITE_PNG_COMPRESSION, 0])

0it [00:00, ?it/s]

# Task B Impainting

## TELEA and NS

In [15]:
impainted = []
for i, img in tqdm(enumerate(S2)):
    impainted.append(cv2.inpaint(img.copy(), Masks[i].copy(), 3, cv2.INPAINT_TELEA))
    cv2.imwrite("Output/Task B TELEA "+ id_to_Scene[i] +".png", impainted[-1], [cv2.IMWRITE_PNG_COMPRESSION, 0])

    impainted.append(cv2.inpaint(img.copy(), Masks[i].copy(), 3, cv2.INPAINT_NS))
    cv2.imwrite("Output/Task B NS "+ id_to_Scene[i] +".png", impainted[-1], [cv2.IMWRITE_PNG_COMPRESSION, 0])

0it [00:00, ?it/s]

# Task B Result Comparison

In [17]:
Results = {"SSD": [], "MSE": []}

Results["SSD"] = [SSD(impaint.copy(), S1[i//2].copy()) for i, impaint in enumerate(impainted)]
Results["MSE"] = [MSE(impaint.copy(), S1[i//2].copy()) for i, impaint in enumerate(impainted)]

In [18]:
print(Results)


{'SSD': [68474650, 67278149, 70018346, 70368991, 62173368, 62746952, 95087978, 95041071, 42098173, 42367291, 136203249, 136311655], 'MSE': [24.76658347800926, 24.333821252893518, 25.32492259837963, 25.451747323495372, 22.487473958333332, 22.694933449074075, 34.39235315393518, 34.37538736979167, 15.226480396412038, 15.32381763599537, 49.26332790798611, 49.302537254050925]}


# Background Subtraction

In [19]:

objects = []

blank = cv2.imread("Blank_Background.jpeg",0)


for subfile in os.listdir("Background Images"):
    
    backgrounds = []
    
    backSubMOG2 = cv2.createBackgroundSubtractorMOG2()
    
    for image in os.listdir("Background Images/"+subfile):
    
        image = cv2.imread("Background Images/"+subfile+"/"+image, 0)
        
        MOG2Mask = backSubMOG2.apply(image)
        
    
    cv2.imshow("Foreground Mask", MOG2Mask)
    cv2.waitKey(0)
    
    cv2.imwrite("Background Changes Visualisation/"+subfile+".png", MOG2Mask)

In [20]:

objects = []

blank = cv2.imread("Blank_Background.jpeg",0)


for subfile in os.listdir("Background Images"):
    
    backgrounds = []
    
    backSubMOG2 = cv2.createBackgroundSubtractorMOG2()
    
    for image in os.listdir("Background Images/"+subfile):
    
        image = cv2.imread("Background Images/"+subfile+"/"+image, 0)
        
        MOG2Mask = backSubMOG2.apply(image)
        
    
    cv2.imshow("Foreground Mask", MOG2Mask)
    cv2.waitKey(0)
    
    cv2.imwrite("Background Changes Visualisation/"+subfile+".png", MOG2Mask)